# Hashi - An Integer Programming Solution

The following notebook details the integer programming solution to the game of Hashi. What follows is a function that utilises NetworkX to display the Hashi boards, and a function that programs our IP model using the optimization software Gurobi. To find the solution for a board please refer to the `Test Boards` and copy the required variables into the given cellblock.

For completeness, we present the Hashi model below. For further details on the decision variables and parameters used, refer to the report.

\begin{alignat}{3}
        \text{ minimise } & \mathbf{0}^{T}\mathbf{x} \label{objectiveHashi}\\
        \text{ subject to } & \sum_{\substack{a \in \lambda \\ \lambda \in B}}         x^{1}_{a} + 2 x^{2}_{a} \hspace{2mm} = \hspace{2mm} L_{a}, && \quad             \forall a \in \{0 , ..., k-1\}  \\
        & x^{0}_{q} + x^{1}_{q} + x^{2}_{q} \ = \ 1, \hspace{2mm} && \quad             \forall q \in B  \\
        & x^{0}_{q} + x^{0}_r \hspace{2mm} \geq \hspace{1mm} 1, && \quad               \forall q,r \in B \hspace{2mm} \\
        &&& \quad\text{ s.t. } o_{q} = 0, \hspace{1mm} o_{r} = 1,  \notag \\
        &&& \quad a \neq b \neq c \neq d, \notag \\
        &&& \quad a > c, \notag \\
        &&& \quad b < d, \notag \\
        &&& \quad j_{a} < j_{c}, \notag \\
        &&& \quad j_{b} > j_{c}. \notag \\
        & \sum_{q \in A} y_{q} - \sum_{q' \in A} y_{q'} = S_{a}, && \quad               \forall a \in \{0, ..., k-1\} \\
        & y_{q} + y_{q'} \leq M(1 - x_{q}^{0}), && \quad \forall q \in B \\
        & M(y_{q} + y_{q'}) \geq 1 - x_{q}^{0}, && \quad \forall q \in B \\
        & x^{n}_{q} \in \{0,1\}, && \quad \forall q \in B, \\
        & y_{q} \in \mathbb{R}^{+}, && \quad \forall q \in A \\
        & o_{i} \in \{0,1\}, && \quad \forall i \in k.        
\end{alignat}

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def display_puzzle(P, L, k, m, n, singles, doubles):
    
    G = nx.MultiGraph()
    pos = {}
    V = list(range(k))
    
    for i in range(k):
        row, col = P[i]
        pos[i] = (col, -row)
    
    plt.figure(figsize = (n,m))
    ax = plt.gca()
    
    # Grid
    for i in range(n +2):
        for j in range(m +2):
            rect = plt.Rectangle((j - 1, -i ), 1, 1, facecolor="white", edgecolor="mistyrose", linewidth=1)
            ax.add_patch(rect)
    
    plt.xlim(-1, m)
    plt.ylim(-n, 1)
    plt.gca().set_aspect('equal', adjustable='box')
    
    # Remove x and y ticks and labels
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.set_xticklabels([])  
    ax.set_yticklabels([])  
    
    # Nodes
    options = {"edgecolors":"black", "node_size":1000}
    nx.draw_networkx_nodes(G, pos, nodelist=V, node_color="white", linewidths = 1, **options)
    
    # Labels
    labels = {i : L[i] for i in range(k)}
    nx.draw_networkx_labels(G, pos, labels, font_size=20, font_color="black", font_weight = "bold",
                            font_family = "Verdana")
    
    # Edges
    if len(singles) != 0:
        nx.draw_networkx_edges(G, pos, edgelist = singles, width = 2)
        
    if len(doubles) != 0:
        for j in range(len(doubles)):
            b = doubles[j]
            pos_u, pos_v = pos[b[0]], pos[b[1]]
            offset = 0.06
        
            if dO[j] == 0:
            # Horizontal doubles
                pos_u_offset1 = (pos_u[0], pos_u[1] + offset)
                pos_v_offset1 = (pos_v[0], pos_v[1] + offset)
                pos_u_offset2 = (pos_u[0], pos_u[1] - offset)
                pos_v_offset2 = (pos_v[0], pos_v[1] - offset)
            
                pos[b[0]] = pos_u_offset1
                pos[b[1]] = pos_v_offset1
                nx.draw_networkx_edges(G, pos, edgelist= [b], width = 2, edge_color = 'black')
            
                pos[b[0]] = pos_u_offset2
                pos[b[1]] = pos_v_offset2
                nx.draw_networkx_edges(G, pos, edgelist= [b], width = 2, edge_color = 'black')

            else:
            # Vertical doubles
                pos_u_offset1 = (pos_u[0] + offset, pos_u[1])
                pos_v_offset1 = (pos_v[0] + offset, pos_v[1])
                pos_u_offset2 = (pos_u[0] - offset, pos_u[1])
                pos_v_offset2 = (pos_v[0] - offset, pos_v[1])
            
                pos[b[0]] = pos_u_offset1
                pos[b[1]] = pos_v_offset1
                nx.draw_networkx_edges(G, pos, edgelist= [b], width = 2, edge_color = 'black')
        
                pos[b[0]] = pos_u_offset2
                pos[b[1]] = pos_v_offset2
                nx.draw_networkx_edges(G, pos, edgelist= [b], width = 2, edge_color = 'black')
    
    if len(doubles) == 0 and len(singles) == 0:
        plt.savefig(f"puzzle_plot_start_grid_{k}_{n}_{m}.png", format='png')
    else:
        plt.savefig(f"puzzle_plot_solution_{k}_{n}_{m}.png", format='png')
        
    plt.show()

In [ ]:
from gurobipy import*
import gurobipy as gp
from gurobipy import GRB 
import time

# Define the set of possible bridges
def create_B(k, P):
    B = []
    # Iterating over the islands
    for a in range(k):
        # Travelling right - the adjacent number if it is in the same line
        if a < k-1 and P[a][0] == P[a+1][0]:
            B.append((a, a+1))
    
        # Travelling down - the island with the same j and next largest i
        V = False
        for b in range(a+1, k):
            if V == False and P[a][1] == P[b][1]:
                B.append((a, b))
                V = True
    return B

# Define the set of arcs (i.e. possible bridges in both directions)
def create_A(B):
    A = []
    # Iterating over the set of possible bridges, adding both directions to A
    for q in B:
        i, j = q
        A.append(q)
        A.append((j,i))
    
    return A

# determining the orientation of the possible bridges
# 0 for horizontal, 1 for vertical
def orientation(B, I):
    O = []
    for q in B:
        a = q[0]
        b = q[1]
        i_a, j_a = P[a]
        i_b, j_b = P[b]
        
        # horizontal
        if i_a == i_b:
            O.append(0)
        else:
            O.append(1)
    
    return O   

# Find the arcs into a given island
def arcs_of_island(i, A):
    arcs_into = []
    arcs_outof = []
    
    for a in A:
        if i == a[0]:
            arcs_outof.append(a)
        if i == a[1]:
            arcs_into.append(a)
    
    return (arcs_into, arcs_outof)

def B_a(B, a):
    B_a = []
    c = 0
    
    for q in B:
        if c < 4:
            if a == q[0] or a == q[1]:
                B_a.append(q)
                c += 1
        else:
            break
    return B_a


def Solve_Hashi(P, L, k):
    model = gp.Model('Hashi')
    model.setParam('Presolve', 0)
    model.setParam('OutputFlag', 0)
    
    B = create_B(k, P)
    A = create_A(B)
    O = orientation(B,P)
    
    print('This Hashi board has', k, 'islands, and', len(B),'possible bridges.')
    print(' ')
    
    # Defining the decision variables X_n, to determine what bridge exists 
    # between islands where a bridge is possible
    X_0 = model.addVars(B, vtype = GRB.BINARY, name = "X_0")
    X_1 = model.addVars(B, vtype = GRB.BINARY, name = "X_1")
    X_2 = model.addVars(B, vtype = GRB.BINARY, name = "X_2")
    
    # Define decision vaiable y, for arc flow
    y = model.addVars(A, vtype = GRB.INTEGER, name = "y")
    
    model.update()
    
    ## BRIDGE INTERSECTION CONSTRAINT
    
    # Looping over all bridge combinations - one way due to symmetry
    l = len(B)
    for AB in range(l):
        for CD in range(AB, l):

            # If they have different orientations
            if O[AB] != O[CD]:
                if O[AB] == 0:
                    a,b = B[AB]
                    c,d = B[CD]
                else:
                    a,b = B[CD]
                    c,d = B[AB]

                i_a, j_a = P[a]
                i_b, j_b = P[b]
                i_c, j_c = P[c]

                # Conditions for intersection - split for ease of logic
                if a != c and a != d and b != c and b != d:
                    if a > c and b < d:
                        if j_a < j_c and j_b > j_c:
                            model.addConstr(X_0[a,b] + X_0[c,d] >= 1, name = f"intersection_constr_{a,b}_{c,d}")
    
    ## AT MOST TWO BRIDGES BETWEEN ISLANDS CONSTRAINT
    for q in B:
        model.addConstr(X_0[q] + X_1[q] + X_2[q] == 1, name = f"bridge_constr_{q}")
   
    ## CONNECTED SYSTEM CONSTRAINT
    M = 126        

    # Define the supply of each island, where the source island, index 0, 
    # has a supply of k-1 and the rest have supply -1
    S = [k-1]
    for i in range(k-1):
        S.append(-1)
    
    # Constrain that the supply is matched for each island
    for i in range(k):
        # Find the arcs into and outof island i
        arcs_into_i, arcs_outof_i = arcs_of_island(i, A)

        model.addConstr(gp.quicksum(y[arc] for arc in arcs_outof_i) - gp.quicksum(y[arc] for arc in arcs_into_i) == S[i], name = f"supply_constr_{i}")

    # Big-M constraints to ensure alignment between the arcs and bridge variables
    for q in B:
        q_rev = (q[1],q[0])

        model.addConstr(y[q] + y[q_rev] <= M*(1 - X_0[q]), name = f"big_m_constr_1_{q}")
        model.addConstr(y[q] + y[q_rev] >= 1 - X_0[q], name = f"big_m_constr_2_{q}")
        
    ## ISLAND LABELS CONSTRAINT
    for a in range(k):
        poss_a = B_a(B,a)
        model.addConstr(gp.quicksum(X_1[brdg] + 2*X_2[brdg] for brdg in poss_a) == L[a], name = f"label_constr_{a}")

    model.update()
    print('This model is then summarised as follows:')
    print(model)
    print(' ')
    
    # OPTIMIZE MODEL
    start = time.time()
    model.optimize()
    end = time.time()

    elapsed = end - start
    print('The time taken to find the solution was',elapsed,' seconds.')
    
    # For solution board
    singles = []
    doubles = []
    for v in model.getVars():
        if v.varname.startswith('X_1') and v.x == 1.0:
            parts = v.varname.split('[')
            indices_str = parts[1].split(']')[0]  # Get the part between the brackets
            singles.append(tuple(map(int, indices_str.split(','))))

        if v.varname.startswith('X_2') and v.x == 1.0:
            parts = v.varname.split('[')
            indices_str = parts[1].split(']')[0]  # Get the part between the brackets
            doubles.append(tuple(map(int, indices_str.split(','))))

    return(singles, doubles)

In [ ]:
# Paste here the board from 'Test Boards'
# P, L, k, m, n

In [ ]:
# Display initial puzzle grid 
display_puzzle(P, L, k, m, n, singles = [], doubles = [])

In [ ]:
# Run optimization
singles, doubles = Solve_Hashi(P, L, k)

In [ ]:
# Display solution grid
dO = orientation(doubles, P)
display_puzzle(P, L, k, m, n, singles, doubles)

# 